In [15]:
import os
import plotly.graph_objects as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

from sqlalchemy import create_engine

# Import environment variables
from dotenv import load_dotenv
load_dotenv()

# Read data from database and order by table_order and result_order and convert date_sampled to string '%m/%d/%Y'
engine = create_engine(os.getenv("DB_CONNECTION"))
df = pd.read_sql(f"SELECT * FROM {os.getenv('DB_TABLE')} ORDER BY table_order, result_order", engine)
df['date_sampled'] = pd.to_datetime(df['date_sampled']).dt.strftime('%m/%d/%Y')
engine.dispose()

In [18]:
# Configurable column names
client_id_column = 'client_id'
concentration_column = 'conc_rl'
analyte_column = 'analyte'
analyte_group_column = 'analyte_group'
mdl_column = 'mdl'
units_column = 'units'
location_id_column = 'location_id'
detected_column = 'detected'
dataset_column = 'title'

# Hover columns and configuration
hover_columns = ['lab_id', 'date_sampled', 'matrix', 'mdl', 'validator_qualifier', 'reason_code']
hover_config = (
    '<b>Client ID:</b> %{x}<br>'
    '<b>Lab ID:</b> %{customdata[0]}<br>'
    '<b>Date Sampled:</b> %{customdata[1]}<br>'    
    '<b>Matrix:</b> %{customdata[2]}<br>'
    '<b>Concentration:</b> %{y}<br>'
    '<b>MDL:</b> %{customdata[3]}<br>'
    '<b>Validator Qualifiers:</b> %{customdata[4]}<br>'
    '<b>Reason Codes:</b> %{customdata[5]}<extra></extra>'
)

# Screening level columns
screening_columns = ['risk_based_action_limit_rsl', 'risk_based_action_limit_ecological_screening', 'project_action_level', 'tclp_limit', 'federal_and_ohio_mcl_action_limit', 'risk_based_action_limit_tapwater_rsl', 'risk_based_action_limit_ohio_swqc']

# Create the dataset dropdown widget and retain the order of values in the original dataframe
dataset_options = df[dataset_column].unique()
dataset_dropdown = widgets.Dropdown(
    options=dataset_options,
    value=dataset_options[0],
    description='Dataset:',
)

# Creating the location_id dropdown widget
location_id_dropdown = widgets.Dropdown(
    description='Location ID:',
    disabled=False,
)

# Initialize empty plot
fig = go.FigureWidget()
fig.update_layout(
    xaxis_title='Client ID',
    yaxis_title='Concentration',
)

# Function to update the location_id dropdown based on selected dataset
def update_based_on_dataset(*args):
    dataset_filtered_df = df[df[dataset_column] == dataset_dropdown.value]
    
    # Update location_id dropdown options
    location_ids = dataset_filtered_df[location_id_column].unique()
    location_id_dropdown.options = location_ids
    if location_ids.size > 0:
        location_id_dropdown.value = location_ids[0]
        update_based_on_location()

# Function to update the plot and dropdowns based on selected location_id
def update_based_on_location(*args):
    location_filtered_df = df[df[location_id_column] == location_id_dropdown.value]
    
    # Update analyte group dropdown options
    analyte_groups = location_filtered_df[analyte_group_column].unique()
    analyte_group_dropdown.options = analyte_groups
    if analyte_groups.size > 0:
        analyte_group_dropdown.value = analyte_groups[0]
        update_analyte_dropdown()

# Function to update the analyte dropdown based on selected analyte group
def update_analyte_dropdown(*args):
    group_filtered_df = df[(df[location_id_column] == location_id_dropdown.value) & 
                           (df[analyte_group_column] == analyte_group_dropdown.value)]
    
    # Update analyte dropdown options
    analytes = group_filtered_df[analyte_column].unique()
    analyte_dropdown.options = analytes
    if analytes.size > 0:
        analyte_dropdown.value = analytes[0]
        update_plot()
        
# Create checkboxes for each screening level
for column in screening_columns:
    exec(f"{column}_checkbox = widgets.Checkbox(description='{column}', value=False)")

# Function to update the plot based on selected analyte
def update_plot(*args):
    plot_filtered_df = df[
        (df[dataset_column] == dataset_dropdown.value) &
        (df[location_id_column] == location_id_dropdown.value) & 
        (df[analyte_group_column] == analyte_group_dropdown.value) & 
        (df[analyte_column] == analyte_dropdown.value)]
    
    # Extract unit and plot data
    if not plot_filtered_df.empty:
        unit = plot_filtered_df[units_column].iloc[0]
        fig.data = []
        fig.layout.shapes = []
        fig.layout.annotations = []

        # Create masks for detected and not detected
        detected_mask = plot_filtered_df[detected_column]
        not_detected_mask = ~plot_filtered_df[detected_column]

        custom_data = plot_filtered_df[hover_columns]
        hover_template = hover_config

        # Add bar trace for "Detected"
        fig.add_trace(go.Bar(
            x=plot_filtered_df[client_id_column],
            y=plot_filtered_df[concentration_column].where(detected_mask, 0),
            name='Detected',
            marker_color='blue',
            customdata=custom_data,
            hovertemplate=hover_template
        ))

        # Add bar trace for "Not Detected"
        fig.add_trace(go.Bar(
            x=plot_filtered_df[client_id_column],
            y=plot_filtered_df[concentration_column].where(not_detected_mask, 0),
            name='Not Detected',
            marker_color='grey',
            customdata=custom_data,
            hovertemplate=hover_template
        ))

        # Update MDL annotations
        fig.update_traces(text=plot_filtered_df[mdl_column], textposition='outside')
        
        # Add hlines based on checkbox states for each screening level
        for column in screening_columns:
            if eval(f"{column}_checkbox.value"):
                avg_screening_level = plot_filtered_df[column].mean()
                fig.add_hline(y=avg_screening_level, line_dash="dot", 
                            annotation_text=column, annotation_position="bottom right")

        # Update layout
        fig.update_layout(
            title=f'Sample Concentration per Client ID for {analyte_dropdown.value}<br><sub>{location_id_dropdown.value}</sub>',
            xaxis_title='Client ID',
            yaxis_title=f'Concentration ({unit})',
            uniformtext_minsize=8,
            uniformtext_mode='hide',
            showlegend=True,
            barmode='overlay'
        )
    else:
        fig.data = []
        fig.layout.shapes = []
        fig.layout.annotations = []

# Observe changes for each screening level checkbox
for column in screening_columns:
    exec(f"{column}_checkbox.observe(update_plot, names='value')")

# Create analyte group and analyte dropdowns (initially empty)
analyte_group_dropdown = widgets.Dropdown(description='Analyte Group:', disabled=False)
analyte_dropdown = widgets.Dropdown(description='Analyte:', disabled=False)

# Observe changes in dropdowns
dataset_dropdown.observe(update_based_on_dataset, names='value')
location_id_dropdown.observe(update_based_on_location, names='value')
analyte_group_dropdown.observe(update_analyte_dropdown, names='value')
analyte_dropdown.observe(update_plot, names='value')

# Initialize interface
update_based_on_dataset()

# Create checkbox widget
checkboxes = widgets.VBox([eval(f"{column}_checkbox") for column in screening_columns])

# Create download button
download_button = widgets.Button(description='Download CSV', button_style='info')

# Display everything (dropdowns and checkboxes are in two separate columns at the top of the page) and the plot below
display(widgets.HBox([widgets.VBox([dataset_dropdown, location_id_dropdown, analyte_group_dropdown, analyte_dropdown]), checkboxes]))
display(fig)




FigureWidget({
    'data': [{'customdata': array([['240-68590-11', '08/20/2016', 'Surface Soil 0 to 0.5 ft bgs', 5.8,
                                    'J', 'j'],
                                   ['240-68590-12', '08/20/2016', 'Surface Soil 0 to 0.5 ft bgs', 5.8,
                                    'J', 'j'],
                                   ['240-68590-10', '08/20/2016', 'Surface Soil 0 to 0.5 ft bgs', 5.8,
                                    'J', 'j'],
                                   ['240-68971-11', '08/31/2016', 'Surface Soil 0 to 0.5 ft bgs', 5.8,
                                    'J', 'j'],
                                   ['240-68971-12', '08/31/2016', 'Surface Soil 0 to 0.5 ft bgs', 5.8,
                                    'J', 'j'],
                                   ['240-68971-10', '08/31/2016', 'Surface Soil 0 to 0.5 ft bgs', 5.8,
                                    'J', 'j'],
                                   ['240-85477-3', '09/22/2017', 'Surface Soil 0 to 0